In [1]:
from dotenv import load_dotenv
import os
import google.generativeai as genai

/home/supreet/projects/gemini_hands_on/.conda/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
load_dotenv()

True

In [18]:
from typing_extensions import TypedDict

class State(TypedDict):
        query: str
        subqueries: list[str]
        # aggregated_results: list[str]
        # summary: str

In [42]:
from langchain_core.prompts import ChatPromptTemplate

QUERY_PARSING_TEMPLATE= """
        You are an assistant specialized in breaking user queries into smaller, logical subqueries.
        Decompose the following query into distinct, manageable subqueries:

        User Query: "{query}"

        Provide the subqueries as a numbered list, one per line.
        Example input: What is X and how does it affect Y?
        Example output:
        1. What is X?
        2. How does X affect Y?"""
        

parsing_prompt = ChatPromptTemplate.from_template(QUERY_PARSING_TEMPLATE)

In [43]:
from langchain_google_genai import ChatGoogleGenerativeAI
llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash-latest")


In [44]:
import re

def parse_query(state: State)-> State:
    out = llm.invoke(parsing_prompt.format(query=state["query"]))
    subqueries = [re.sub(r"^\d+\.\s+", "", s) for s in out.content.split("\n") if s.strip()]
    return {"subqueries": subqueries}

In [45]:
from langgraph.graph import StateGraph, START, END

graph_builder = StateGraph(State)
graph_builder.add_node("parse", parse_query)
graph_builder.add_edge(START, "parse")

graph = graph_builder.compile()

In [46]:
state = graph.invoke({"query": "What are the latest breakthroughs in AI and what are there ethical implications?"})

print(state)

{'query': 'What are the latest breakthroughs in AI and what are there ethical implications?', 'subqueries': ['What are the latest breakthroughs in AI?', 'What are the ethical implications of the latest breakthroughs in AI?']}
